# Old Wiki Scraper

In [ ]:
# extaract unique social complexity variables we are interested in
# This should be done only once.
import pandas as pd

df = pd.read_excel('Equinox_2020_packaged_xls.xls', sheet_name="Equinox2020_CanonDat")

unique_sc_variables = []
unique_sc_subsections = []
for row in df.iterrows():
    if row[1]['Section'] == "Social Complexity variables":
        if row[1]['Variable'] not in unique_sc_variables:
            unique_sc_variables.append(row[1]['Variable'])
        if row[1]['Subsection'] not in unique_sc_subsections:
            unique_sc_subsections.append(row[1]['Subsection'])

print(unique_sc_variables)
print(len(unique_sc_variables))
print("____")
print(unique_sc_subsections)



# a bettre alternative:
# look at the following function (from main_scraper.py)
# return_all_sc_vars()


In [1]:
from main_scraper import *
from z_helpers import * 


sc_var_html_extractor()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
ultimate_dic = sc_variables_extractor()
pretty_ultimate_dic = all_sc_vars_augmenter(ultimate_dic)
pretty_ultimate_dic_with_conf_disp = add_confidence_disputed_to_augmentd_sc(pretty_ultimate_dic)

******** ra   done and saved as : ra
******** polity_territory   done and saved as : polity_territory
******** polity_population   done and saved as : polity_population
******** population_of_the_largest_settlement   done and saved as : population_of_the_largest_settlement
******** settlement_hierarchy   done and saved as : settlement_hierarchy
******** administrative_levels   done and saved as : administrative_level
******** religious_levels   done and saved as : religious_level
******** military_levels   done and saved as : military_level
******** professional_military_officers   done and saved as : professional_military_officer
******** professional_soldiers   done and saved as : professional_soldier
******** professional_priesthood   done and saved as : professional_priesthood
******** full-time_bureaucrats   done and saved as : full_time_bureaucrat
******** examination_system   done and saved as : examination_system
******** merit_promotion   done and saved as : merit_promotion
**

In [ ]:
# from main_scraper import *
# from z_helpers import * 
# pretty_ultimate_dic_with_conf_disp = {"a": "b"}

In [4]:
get_it_done_sql(pretty_ultimate_dic_with_conf_disp, False)

288 sql insertion rows added to: /home/majid/dev/old_wiki_scraper_for_social_complexity/SC_SQL_AWS/aaaaall_all_all.sql


In [ ]:
pretty_ultimate_dic_with_conf_disp["administrative_level"]

In [ ]:
df = pretty_ultimate_dic_with_conf_disp["population_of_the_largest_settlement"]
cond = df["raw"].str.contains("{")
filtered_df = df[cond]

In [ ]:
def describe_set_of_values_after_pretty(my_ultimate_dic):
    """
    for the after_pretty case
    """
    for key_variable, df_full in my_ultimate_dic.items():
        print(f"*********{key_variable}")
        for inner_var in list(df_full.columns):
            if "XYZ_" in inner_var:
                unique_values = len(df_full[inner_var].unique())
                if unique_values <=50:
                    for value in df_full[inner_var]:
                        if value not in ["present", "absent", "UNKNOWN", "0", "1", "2", "3","4", "5", "6", "7","8", "9", "10", "11", "12", "13"]:
                            print(f"*{value}*")
                    print()
                print("_________________")


In [ ]:

def convert_general_vars_dic_to_two_dfs(my_pretty_dic):
    # let's create the mother meta_dfs we need for taking care of the data read from almost anywhere 
    from sc_definitions import SC_VAR_DEFINITIONS, SC_VAR_SUBSECTIONS
    # meta_cols_in_variables = [
    #     "colname", "dtype", "varname", "units", "min", "max", "scale", 
    #     "decimal_places", "max_digits", "choices", "foreign_key", "foreign_key_related_name", "null_meaning", 
    # ]

    # meta_variables = [
    #     "varname", "db_name", "main_desc", "main_desc_source", "notes", "cols", "section", "subsection", "needsSeshatCommon",
    # ]

    my_vars_dic = {}

    for variable, inner_data in my_pretty_dic.items():
        variable_spaced = variable.replace("_", " ")
        if variable in ["ra"]:
            my_cols = 1
            inner_dic = {
            'col1': {'colname': 'sc_ra',
                'dtype': ['ForeignKey', 'Select'],
                'varname': 'sc_research_assistant',
                'col_exp': 'The RA of Social Complexity Variables',
                'foreign_key': 'Seshat_Expert',
                'foreign_key_related_name': 'sc_research_assistant'}
            }
        elif variable in ["polity_territory", ]:
            my_cols = 2
            inner_dic = {
            'col1': {
                'colname': variable + "_from",
                'dtype': ['IntegerField', 'NumberInput'],
                'varname': variable,
                'col_exp': f'The lower range of {variable_spaced} for a polity.',
                'max_digits': 20,
                'units': 'km squared',},
            'col2': {
                'colname': variable + "_to",
                'dtype': ['IntegerField', 'NumberInput'],
                'varname': variable,
                'col_exp': f'The lower range of {variable_spaced} for a polity.',
                'max_digits': 20,
                'units': 'km squared',
                }
            }
        elif variable in ["polity_population", "population_of_the_largest_settlement", ]:
            my_cols = 2
            inner_dic = {
            'col1': {
                'colname': variable + "_from",
                'dtype': ['IntegerField', 'NumberInput'],
                'varname': variable,
                'col_exp': f'The lower range of {variable_spaced} for a polity.',
                'max_digits': 20},
            'col2': {
                'colname': variable + "_to",
                'dtype': ['IntegerField', 'NumberInput'],
                'varname': variable,
                'col_exp': f'The lower range of {variable_spaced} for a polity.',
                'max_digits': 20,
                }
            }
        elif variable in ["administrative_level", "settlement_hierarchy", "religious_level", "military_level", ]:
            my_cols = 2
            inner_dic = {
            'col1': {
                'colname': variable + "_from",
                'dtype': ['IntegerField', 'NumberInput'],
                'varname': variable,
                'col_exp': f'The lower range of {variable_spaced} for a polity.',
                'max_digits': 2,
                'min': 0
                },
            'col2': {
                'colname': variable + "_to",
                'dtype': ['IntegerField', 'NumberInput'],
                'varname': variable,
                'col_exp': f'The lower range of {variable_spaced} for a polity.',
                'max_digits': 2,
                'min': 0
                }
            }     
        else:
            my_cols = 1
            inner_dic = {
            'col1': {
                'colname': variable,
                'dtype': ['CharField', 'Select'],
                'varname': variable,
                'col_exp': f'The absence or presence of {variable_spaced} for a polity.',
                'choices': "ABSENT_PRESENT_CHOICES",
                }
            }  
        # dic to be appended
        my_var_dic = {
            "varname": variable,
            "db_name": "sc",
            "needsSeshatCommon": True,
            "main_desc": SC_VAR_DEFINITIONS[variable],
            "main_desc_source": "NOTHING",
            "notes": "No_Actual_note",
            "cols": my_cols,
            "section": "Social Complexity",
            "subsection": SC_VAR_SUBSECTIONS[variable],
        }

        my_var_dic.update(inner_dic)
        my_vars_dic[variable] = my_var_dic

    return my_vars_dic

In [ ]:
convert_general_vars_dic_to_two_dfs(pretty_ultimate_dic_with_conf_disp)

In [ ]:
my_tuple_choices = []
str_mid_list = []

for item in ["present", "absent", "unknown"]:
    str_mid_list.append(f"('{item}', '{item}'),\n")
    str_mid = "".join(str_mid_list)
full_string = str_mid 
my_tuple_choices.append(full_string)  

In [ ]:
print(my_tuple_choices)

In [ ]:
pretty_ultimate_dic_with_conf_disp

In [ ]:
my_tuple = pretty_ultimate_dic_with_conf_disp["polity_population"]["raw"]
cond = my_tuple.str.contains("{")
pretty_ultimate_dic_with_conf_disp[cond]

In [ ]:
pretty_ultimate_dic["paper_currency"]["values_df"]

In [ ]:
df = pretty_ultimate_dic["polity_population"]["values_df"]

# df_column_contains_suspected = df["raw"].str.contains("suspected") 
df_column_contains_suspected = df["raw"].str.contains("suspected") 
import numpy as np

new_df = df[df_column_contains_suspected]
df['confidence'] = np.where(df['raw'].isin(['suspected unknown']), 'SSP', 'TRS')

In [ ]:
import numpy as np
df = pd.DataFrame({'raw': ['valid', "another one", 'suspected unknown', 'inferred absent']})
df['confidence'] = np.where(df['raw'].isin(['suspected unknown']), 'SSP', 'TRS')

df['raw'] = df['raw'].str.replace("inferred", "")

In [ ]:
df

In [ ]:
pretty_ultimate_dic["polity_population"]["values_df"][100:140]

In [ ]:
df = pretty_ultimate_dic["mnemonic_devices"]["values_df"]

assert(df['polity'].str.contains('^[a-zA-Z_]+$')).all(), "Error: Polity column contains values that are not solely composed of alphabetical characters and/or underscores."

In [ ]:
MlMali*	

In [ ]:
pretty_ultimate_dic["mnemonic_devices"]["values_df"][100:140] 

In [ ]:
describe_set_of_value_dfs(pretty_ultimate_dic)

In [ ]:
def range_finder_plus_plus_non_numeric(input_str, a_str):
    """
    This will catch:
    
    - absent: 54-56 BCE
    - disputed_present_absent
    - uncertain_absent_present: 546 CE
    """
    stripped_str = input_str.replace(",", "").strip()
    #my_catches = re.findall(r'(\w+)\s*:{0,1}\s*(.*)', stripped_str)
    my_catches = re.findall(r'(present|suspected unknown|NO_VALUE_ON_WIKI|inferred present|absent|inferred absent|unknown|uncertain_present_absent|disputed_present_absent|uncertain_absent_present|disputed_absent_present|disputed_inferred_absent_and_present|disputed_inferred_absent_and_inferred_present|uncertain_absent_and_inferred_present)\s*:{0,1}\s*(.*)', stripped_str)
    if len(my_catches) == 1:
        years_potential = my_catches[0][1]
        extracted_years = is_there_years_here(years_potential)
        range_low = my_catches[0][0]
        if extracted_years != [None, None] and range_low:
            return [range_low, extracted_years[0], extracted_years[1]]
        elif range_low:
            return [range_low, None, None]
    else:
        return [None, None, None]
    
#range_finder_plus_plus_non_numeric("present: 800-1019 CE", ":")
#range_finder_plus_plus_non_numeric("disputed_present_absent", ":")
range_finder_plus_plus_non_numeric("uncertain_absent_present: 700-732 CE", ":")


In [ ]:
a_mapping = get_all_users_experts(pretty_ultimate_dic, 100)

In [ ]:
a_new_dic = {}
for key in a_mapping.keys():
    a_new_dic[key] = key

In [ ]:
#pretty_ultimate_dic["scale_of_supra-cultural_interaction"]["values_df"]
for index, row in ultimate_dic["indigenous_coins"]["values_df"].iterrows():
    if "[absent" in row.wiki_value:
        print(str(row.polity), "---->", row.wiki_value) 

In [ ]:
pretty_ultimate_dic["scale_of_supra-cultural_interaction"]["values_df"]

In [ ]:
wide_df = pretty_ultimate_dic["scale_of_supra-cultural_interaction"]["values_df"]
wide_df.loc[wide_df.polity == 'IsCommw']

In [ ]:
stripped_str = "[9,000,000]: 98-43"
stripped_str = stripped_str.replace(",", "")
a_str = ";"
my_catches = re.findall(fr'[\[]{{0,1}}[ ]{{0,1}}(\d{{1,15}})[ ]{{0,1}}[-|_| ]{{0,1}}[ ]{{0,1}}([\d{{1,15}}]{{0,1}})[ ]{{0,1}}[\]]{{0,1}}[ ]{{0,1}}[{a_str}]{{0,1}}(.*)', stripped_str)

In [ ]:
my_catches

In [ ]:
for index, row in ultimate_dic["ra"]["values_df"].iterrows():
    if "AP" in row[2]:
        print(str(row.polity), "---->", row[2]) 

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))
pd.set_option('display.expand_frame_repr', True)
pd.options.display.max_colwidth = 500

In [ ]:
ultimate_dic.keys()

In [ ]:
expanded_df = ultimate_dic["language_genus"]["values_df"]

In [ ]:
expanded_df.wiki_value.unique()

In [ ]:
print(expanded_df.columns())

In [ ]:
a = {'a': 123, 'b': 897}
print(f"Halolo there: {a}")

In [ ]:
expanded_df.loc[expanded_df.wiki_year_from > 1].head(30)

In [ ]:
expanded_df.loc[expanded_df.wiki_clean != '']

In [ ]:
pretty_ultimate_dic["language_genus"]["values_df"]["wiki_clean"].unique()

<hr>

In [ ]:
thin_dic, new_json_file = general_vars_sql_maker(ultimate_dic, "Majid", "/z_control_file.json", False)

In [ ]:
thin_dic

In [ ]:
import os
print(os.getcwd())

In [ ]:
int(None)

In [ ]:
from datetime import datetime

In [ ]:
a = datetime.today().date()

In [ ]:
a

In [ ]:
describe_set_of_value_dfs(ultimate_dic)

In [ ]:
wide_df = ultimate_dic["scale_of_supra-cultural_interaction"]["values_df"]
long_df = value_semi_colon_expander(wide_df)
#wide_df_alter = ultimate_dic["language_genus"]["values_df"]

#merged_long_dfs = []
long_df_vs = value_semi_colon_splitter_plus(wide_df)
#alternate_long_df_vs = value_semi_colon_splitter_plus(wide_df_alter)
#merged_long_dfs = list(set(long_df_vs + alternate_long_df_vs))

In [ ]:
long_df[0].tail(20)

In [ ]:
long_df_vs

In [ ]:
for my_str in long_df_vs:
    is_there_years_here(my_str)

In [ ]:
wide_df.head(10)

In [ ]:
for item in merged_long_dfs:
    print(f"'{item}'", end=", ")

In [ ]:
for item in alternate_long_df_vs:
    print(f"'{item}'", end=", ")

In [ ]:
for item in long_df_vs:
    print(f"'{item}'", end=", ")

In [ ]:
long_df_vs

In [ ]:
for row, item in expanded_df.iterrows():
    if ';' in item[2]:
        print(item[2])

In [ ]:
wide_df.loc[wide_df.wiki_value == '8th century BCE']

In [ ]:
unique_religions = []
for row, values in ultimate_dic["alternate_religion_genus"]["values_df"].iterrows():
    print(len(values))
    if len(values[2].split(';')) > 1:
        for item in values[2].split(';'):
            if ":" in item.strip():
                double_stripped = item.split(":")[0].strip()
            else:
                double_stripped = item.strip()
            if double_stripped not in unique_religions:
                unique_religions.append(double_stripped)
    else:
        if values[2].strip() not in unique_religions:
            if ":" in values[2].strip():
                double_stripped = values[2].strip().split(":")[0]
            else:
                double_stripped = values[2].strip().strip()
            if double_stripped not in unique_religions:
                unique_religions.append(double_stripped.strip())


In [ ]:
unique_religions

In [ ]:
gen_var_html_extractor()

In [ ]:
import re
my_str = """
Sogdiana and establish a capital city at Pendjikent."[MAJIDBENAM_REF_21]</div><div>Toramana, who made conquests in India "established his capital city at Salaka" [MAJIDBENAM_REF_22]</div><div class='meatypart'><b>♠ Language ♣ Bactrian ♥</b> Bactrian; Iranian; Turkic According to the Chinese records from the period, the language of the Hephthalites was distinct from that of those Iranian-speaking people of Central Asia who were called Hu by the Chinese. However, there is no consensus among scholars.[MAJIDBENAM_REF_23] Recent work has reappraised Chinese manuscript sources to postulate that the Hephtalites had ceased to retain their original Altaic language and adopted Bactrian by the end of the fourth century. [MAJIDBENAM_REF_24] "Probably dominated by an Eastern Iranian language, but their mixed ancestry also lead to multilingualism."[MAJIDBENAM_REF_25]<b>♠ Linguistic family ♣ Indo-European ♥</b></div><div class='meatypart'><b>♠ Religion Genus ♣ Hephthalite Religions ♥</b> "Information about the religion of the Hephthalites is provided by the Chinese sources. Sung Yun reports that in Tokharistan ‘the majority of them do not """
RefRegex = re.compile(r'(<b>♠) Linguistic family(.*?)</b>(.*?)(</div>)')
catches_all = RefRegex.finditer(my_str)
print(catches_all)
if catches_all:
    for index, catches in enumerate(catches_all):
        my_str = my_str.replace(catches.group(1), "</div><div class='meatypart'><b>♠")

In [ ]:
my_str

In [ ]:
def tuple_choices_maker_2(my_new_list_of_choices, choice_name):
    my_tuple_choices = []
    str_mid_list = []
    the_name_of_the_tuple = f"{choice_name.upper()}_CHOICES"
    str_top = f"{the_name_of_the_tuple} = (\n"
    str_bot = ")\n\n"

    for item in my_new_list_of_choices:
        #goodie = v[good_key]["colname"]
        str_mid_list.append(f"('{item}', '{item}'),\n")
        str_mid = "".join(str_mid_list)
    full_string = str_top + str_mid + str_bot
    my_tuple_choices.append(full_string)        
                       
    # take care of beginning and end
    my_tuple_choices.insert(0, f"\n##### tuple choices for  {choice_name}.\n")
    my_tuple_choices_str = "".join(my_tuple_choices)
    return the_name_of_the_tuple, my_tuple_choices_str

degree_of_centralization_CHOICES = ['loose', 'confederated state', 'unitary state', 'nominal', 'confederate state', 'quasi-polity', 'suspected unknown', 'none', 'unknown', 'confederation', 'polity', 'NO_VALUE_ON_WIKI', 'nominal allegiance', 'unitary', ]

my_text = tuple_choices_maker_2(degree_of_centralization_CHOICES, "degree_of_centralization")[0]

In [ ]:
print(my_text)

In [ ]:
from main_scraper import *
import re

my_str = """
  <div><a href="/w/index.php?title=File:Asia_500ad.jpg" class="image"><img alt="Asia 500ad.jpg"
                src="/w/images/thumb/8/87/Asia_500ad.jpg/500px-Asia_500ad.jpg"
                srcset="/w/images/thumb/8/87/Asia_500ad.jpg/750px-Asia_500ad.jpg 1.5x, /w/images/thumb/8/87/Asia_500ad.jpg/1000px-Asia_500ad.jpg 2x"
                width="500" height="284"></a></div>"""
after_img = image_remover(my_str)

In [ ]:
after_img

In [ ]:
print(catches.group(1))

In [ ]:
a = """<p><br>
</p>"""
print(a.replace("\n", ""))

In [ ]:
big_dic_1 = general_variables_extractor()

In [ ]:
len(big_dic_1)

In [ ]:
big_dic_1["Degree of centralization"]

In [ ]:
import pandas as pd

df = pd.read_excel('Equinox_2020_packaged_xls.xls', sheet_name="Equinox2020_CanonDat")



In [ ]:
len(df)

In [ ]:
unique_general_variables = []
for row in df.iterrows():
    if row[1]['Section'] == "General variables":
        if row[1]['Variable'] not in unique_general_variables:
            unique_general_variables.append(row[1]['Variable'])

In [ ]:
print(unique_general_variables)

In [ ]:
import re

def is_there_years_here(my_str):
    my_catches = re.findall('(-?\d{1,4})[ ]{0,1}([B|b]*[C|c][E|e]*)*[-|_| ](\d{1,4})[ ]{0,1}([B|b]*[C|c][E|e]*)*', my_str)
    if len(my_catches) > 1:
        print(f"Catches are more than 1: {len(my_catches)}")
    return my_catches

def remove_year_range(a_str):
    my_new_str = re.sub('(-?\d{1,4})[ ]{0,1}(B*CE*)*[-|_| ](\d{1,4})[ ]{0,1}(B*CE*)*', "", a_str)
    while "  " in my_new_str:
        my_new_str  = my_new_str.replace("  ", " ").strip()
    return my_new_str

my_str = "hi 750-54 Bc ali"

my_catch = is_there_years_here(my_str)
b = remove_year_range(my_str)

In [ ]:
my_catch

In [ ]:
new_str = my_str
for item in my_catch:
    new_str = new_str.replace(item.group(1), "")
    print(item.group(1))
print(my_str)
print(new_str)

In [ ]:
stripped_str = "[3000000-4000000] tt4 "
my_catches = re.findall('\[[ ]{0,1}(\d{1,15})[ ]{0,1}[-|_| ][ ]{0,1}(\d{1,15})[ ]{0,1}\]', stripped_str)

In [ ]:
len(my_catches)

In [ ]:
def utc_zone_finder(a_str, a_polity):
    stripped_str = a_str.replace(",", "").strip()
    letter_part_obj = re.search('([A-Z])', stripped_str)
    num_part_obj = re.search('(\d{1,2})', stripped_str)
    num_part = num_part_obj.group(0)
    letter_part = letter_part_obj.group(0)
    if num_part and letter_part:
        output = f"{num_part} {letter_part}"
    else:
        output = "NO_VALID_VALUE"
        print(f"BAD_VALUE_GIVEN: {a_str} for: {a_polity}")
    return output

print(utc_zone_finder("45 R", "Ali"))

In [ ]:
MYINT = ['IntegerField', 'NumberInput']
MYDEC = ['DecimalField', 'NumberInput']
MYTXT = ['CharField', 'TextInput']
# for these we need, the pandas dfs to smartly select the choices, put them in a tuple or list
MYTXT_CH = ['CharField', 'Select']
MYFOREIGN = ['ForeignKey', 'Select']


def tuple_choices_maker_2(my_new_list_of_choices, choice_name):
    my_tuple_choices = []
    str_mid_list = []
    the_name_of_the_tuple = f"{choice_name.upper()}_CHOICES"
    str_top = f"{the_name_of_the_tuple} = (\n"
    str_bot = ")\n\n"

    for item in my_new_list_of_choices:
        #goodie = v[good_key]["colname"]
        str_mid_list.append(f"('{item}', '{item}'),\n")
        str_mid = "".join(str_mid_list)
    full_string = str_top + str_mid + str_bot
    my_tuple_choices.append(full_string)        
                       
    # take care of beginning and end
    my_tuple_choices.insert(0, f"\n##### tuple choices for  {choice_name}.\n")
    my_tuple_choices_str = "".join(my_tuple_choices)
    return the_name_of_the_tuple, my_tuple_choices_str

'continuity', 'continuation', 'Continuation', 'contination', 'Continuity', 'continunity', 


degree_of_centralization_CHOICES = ['loose', 'confederated state', 'unitary state', 'nominal', 'confederate state', 'quasi-polity', 'suspected unknown', 'none', 'unknown', 'confederation', 'polity', 'NO_VALUE_ON_WIKI', 'nominal allegiance', 'unitary', ]

supra_polity_relations_CHOICES =  ['none', 'vassalage', 'alliance', 'nominal allegiance', 'suspected unknown', 'nominal', 'personal union', 'NO_VALUE_ON_WIKI', 'unknown', 'Nominal', 'Alliance', 'uncoded',]

language_CHOICES = ['Pashto', 'Persian', 'Greek', 'Bactrian', 'Sogdian', 'Pahlavi', 'Brahmi', 'Kharoshthi', 'Tocharian', 'Chinese', 'archaic Chinese', 'Xiangxi', 'Qiandong', 'Chuanqiandian', 'Hmong-Mien', 'Hmongic', 'Middle Chinese', 'Jurchen', 'Khitan', 'Xianbei', 'Manchu language', 'Mongolian language', 'Atanque', 'Shuar', 'Arabic', 'suspected unknown', 'NO_VALUE_ON_WIKI', 'Demotic', 'Ancient Egyptian', 'Late Egyptian', 'demotic Egyptian', 'Castilian Spanish', 'Chuukese', 'French', 'Langues dOil', 'Occitan', 'Latin', 'Old Frankish', 'Germanic', 'Gallic', 'Gaulish', 'English', 'Akan', 'Twi', 'Doric Greek', 'Minoan', 'Early Greek', 'Eteocretan', 'Old Hawaiian', 'Hawaiian', 'Iban', 'Sanskrit', 'Old Javanese', 'Middle Javanese', 'Javanese', 'Canaanite', 'Aramaic', 'Hebrew', 'Kannada', 'Urdu', 'A’chik', 'Prakrit', 'Telugu', 'Tamil', 'Akkadian', 'Sumerian', 'Amorite', 'Old Babylonian', 'Mesopotamian Religions', 'Old Persian', 'Elamite', 'Egyptian', 'Old Elamite', 'Mongolian', 'native Iranian languages', 'Turkic', 'Turkish', 'Babylonian', 'Hurrian', 'Proto-Elamite', 'Old Norse', 'Italian', 'Middle Japanese', 'Old Japanese', 'Late Old Japanese', 'Japanese', 'Early Modern Japanese', 'Old Turkic', 'Iranian', 'Old Khmer', 'Mon', 'Tai', 'Khmer', 'Pali', 'Phoenician', 'Berber', 'Spanish', 'Portuguese', 'Bambara', 'Mande', 'Songhay', 'Russian', 'Georgian', 'Armenian', 'Kereid', 'Tatar', 'Naimans', 'Khalkha', 'Rouran', 'Xiongnu', 'Oirat', 'Zapotec', 'Icelandic', 'Aymara', 'Puquina', 'Quechua', 'Orokaiva', 'unknown', 'Sindhi', 'Punjabi', 'Sakha (Yakut)', 'Merotic', 'Coptic', 'Thai', 'Proto-Indo-European language', 'Nesite', 'Luwian', 'Hattic', 'Hittite', 'Old Assyrian dialect of Akkadian', 'Indo-European language', 'Lydian', 'Ottoman Turkish', 'Phrygian', 'Miami Illinois', 'Cayuga', 'Mohawk', 'Oneida', 'Onondaga', 'Seneca', 'Tuscarora', 'Middle Mongolian', 'Ancient Iranian', 'Chagatai Turkish', 'Sabaic', 'Mainic', 'Qatabanic', 'Hadramawtic', 'Old Arabic']

# Done Merged (language_genus and linguistic_family)
linguistic_family_CHOICES = ['Indo-European', 'Sino-Tibetan', 'NO_VALUE_ON_WIKI', 'Tungusic', 'Altaic', 'Mongolic', 'Chibcha', 'Chicham', 'Afro-Asiatic', 'Oceanic-Austronesian', 'Celtic', 'Niger-Congo', 'Kwa', 'Hamito-Semitic', 'Austronesian', 'Malayo-Polynesian', 'Semitic', 'Indo-Iranian', 'Dravidian', 'isolate language', 'West Semetic', 'isolate', 'suspected unknown', 'language isolate', 'none', 'Germanic', 'Japonic', 'Turkic', 'Austro-Asiatic, Mon-Khmer', 'Austro-Asiatic', 'unknown', 'Mande', 'Songhay', 'Oghuz', 'Kartvelian', 'Manchu-Tungusic', 'Proto-Mongolic', 'Otomanguean', 'Proto-Otomanguean', 'Mixe-Zoquean', 'Aymaran', 'Quechuan', 'Papuan Languages', 'Tai-Kadai', 'Algonquian', 'Iroquois', 'Iranian']

# Done Merged
religion_genus_CHOICES = ['Zoroastrianism', 'Graeco-Bactrian Religions', 'Buddhism', 'Christianity', 'Islam', 'Mongolian Shamanism', 'Hittite Religions', 'Ismaili', 'Lydian Religions', 'Chinese State Religion', 'Egyptian Religions', 'Ancient Iranian Religions', 'Hellenistic Religions', 'Hephthalite Religions', 'Manichaeism', 'Ancient East Asian Religion', 'Jain Traditions', 'Xiongnu Religions', 'Roman State Religions', 'Shinto', 'Phrygian Religions', 'Mesopotamian Religions', 'Hinduism', 'Ancient Javanese Religions', 'Confucianism',]

# Done Merged
religion_family_CHOICES = ['Saivist Traditions', 'Assyrian Religions', 'Republican Religions', 'Imperial Confucian Traditions', 'Shii', 'Bhagavatist Traditions', 'Sunni', 'Vedist Traditions', 'Saivist', 'Islam', 'Chinese Folk Religion', 'Semitic', 'Vaisnava Traditions', 'Ptolemaic Religion', 'Vedic Traditions', 'Japanese Buddhism', 'Orthodox', 'Vaishnava Traditions', 'Shang Religion', 'Atenism', 'Mahayana', 'suspected unknown', 'Japanese State Shinto', 'Saiva Traditions', 'Sufi', 'Chinese Buddhist Traditions', 'Arian', 'Shia', 'Catholic', 'Western Zhou Religion', 'Imperial Cult', 'Theravada', 'Seleucid Religion',]

# Done Merged
religion_CHOICES = ['Islam', 'Shadhil', 'Karrami', 'Hanafi', 'Mevlevi', 'Ismaili', 'Shafii', 'Shia', 'Twelver', 'Byzantine Orthodox', 'Bektasi', 'NO_VALUE_ON_WIKI', 'Sunni', 'Roman Catholic', ]

relationship_to_preceding_entity_CHOICES = ['continuity', 'elite migration', 'cultural assimilation', 'continuation', 'indigenous revolt', 'replacement', 'population migration', 'hostile', 'disruption/continuity', 'continuity/discontinuity', 'NO_VALUE_ON_WIKI', 'suspected unknown', 'vassalage', 'not applicable',]


ALL_GENERAL_VARS_LIST =[
    {
    'varname': 'polityresearchassistant',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The RA(s) who worked on a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "Staff", # Hier 
    },
    {
    'varname': 'polityutmzone',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The UTM Zone of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityoriginalname',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The original name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternativename',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternative name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politypeakyears',
    'cols': 2,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternative name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityduration',
    'cols': 2,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternative name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politydegreeofcentralization',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The degree of centralization of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politysuprapolityrelations',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The supra polity relations of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politycapital',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The capital or the largest settlement of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politylanguage',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The language or the largest settlement of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politylinguisticfamily',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The linguistic family of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politylanguagegenus',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The language genus of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligiongenus',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The religion genus of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligionfamily',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The religion family of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligion',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The religion of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityrelationshiptoprecedingentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The polity relationship to preceding (quasi)polity", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityprecedingentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The preceding entity of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politysucceedingentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The succeeding entity of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politysupraculturalentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The supracultural entity of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityscaleofsupraculturalinteraction',
    'cols': 2,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The scale_of_supra_cultural_interaction of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternatereligiongenus',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternate religion genus of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternatereligionfamily',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternate religion family of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternatereligion',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternate religion  of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityexpert',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The expert of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityeditor',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The editor of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligioustradition',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The details of religious traditions.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    }
]


ALL_GENERAL_COLS_LIST =[
{
    'colname': "polity_ra",
    'dtype': MYFOREIGN,
    'varname': 'polityresearchassistant', # key
    'col_exp': "The RA of a polity.",
    'foreign_key': "SeshatExpert",
    'foreign_key_related_name': "seshat_research_assistant",
    },
    {
    'colname': "utm_zone",
    'dtype': MYTXT,
    'varname': 'polityutmzone', # key
    'col_exp': "The details of UTM_ZONE.",
    'max_digits': 5,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "original_name",
    'dtype': MYTXT,
    'varname': 'polityoriginalname', # key
    'col_exp': "The details of original_name.",
    'max_digits': 100,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "alternative_name",
    'dtype': MYTXT,
    'varname': 'polityalternativename', # key
    'col_exp': "The details of alternative_name.",
    'max_digits': 100,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "peak_year_from",
    'dtype': MYINT,
    'varname': 'politypeakyears', # key
    'col_exp': "The beginning of the peak year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "peak_year_to",
    'dtype': MYINT,
    'varname': 'politypeakyears', # key
    'col_exp':  "The end of the peak year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "polity_year_from",
    'dtype': MYINT,
    'varname': 'polityduration', # key
    'col_exp': "The beginning year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "polity_year_to",
    'dtype': MYINT,
    'varname': 'polityduration', # key
    'col_exp':  "The end year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "degree_of_centralization",
    'dtype': MYTXT_CH,
    'varname': 'politydegreeofcentralization', # key
    'col_exp': "The details of degree_of_centralization.",
    'max_digits': 50,
    'choices': tuple_choices_maker_2(degree_of_centralization_CHOICES, "degree_of_centralization")[0],
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "supra_polity_relations",
    'dtype': MYTXT_CH,
    'varname': 'politysuprapolityrelations', # key
    'col_exp': "The details of supra polity relations.",
    'max_digits': 50,
    'choices': tuple_choices_maker_2(supra_polity_relations_CHOICES, "supra_polity_relations")[0],
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "capital",
    'dtype': MYTXT,
    'varname': 'politycapital', # key
    'col_exp': "The capital or the largest settlement of a polity.",
    'max_digits': 70,
    'null_meaning': "This polity did not have a capital or the largest settlement."
    },
    {
    'colname': "language",
    'dtype': MYTXT_CH,
    'varname': 'politylanguage', # key
    'col_exp': "The language of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(language_CHOICES, "language")[0],
    'null_meaning': "This polity did not have a language."
    },
    {
    'colname': "linguistic_family",
    'dtype': MYTXT_CH,
    'varname': 'politylinguisticfamily', # key
    'col_exp': "The linguistic family of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(linguistic_family_CHOICES, "linguistic_family")[0],
    'null_meaning': "This polity did not have a linguistic family."
    },
    {
    'colname': "language_genus",
    'dtype': MYTXT_CH,
    'varname': 'politylanguagegenus', # key
    'col_exp': "The language genus of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(linguistic_family_CHOICES, "linguistic_family")[0],
    'null_meaning': "This polity did not have a language Genus."
    },
    {
    'colname': "religion_genus",
    'dtype': MYTXT_CH,
    'varname': 'polityreligiongenus', # key
    'col_exp': "The religion genus of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_genus_CHOICES, "religion_genus")[0],
    'null_meaning': "This polity did not have a religion genus."
    },
    {
    'colname': "religion_family",
    'dtype': MYTXT_CH,
    'varname': 'polityreligionfamily', # key
    'col_exp': "The religion family of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_family_CHOICES, "religion_family")[0],
    'null_meaning': "This polity did not have a religion family."
    },
    {
    'colname': "religion",
    'dtype': MYTXT_CH,
    'varname': 'polityreligion', # key
    'col_exp': "The religion of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_CHOICES, "religion")[0],
    'null_meaning': "This polity did not have a religion."
    },
    {
    'colname': "relationship_to_preceding_entity",
    'dtype': MYTXT_CH,
    'varname': 'polityrelationshiptoprecedingentity', # key
    'col_exp': "The polity relationship to preceding (quasi)polity",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(relationship_to_preceding_entity_CHOICES, "relationship_to_preceding_entity")[0],
    'null_meaning': "This polity did not have a relationship to preceding (quasi)polity"
    },
    {
    'colname': "preceding_entity",
    'dtype': MYTXT,
    'varname': 'polityprecedingentity', # key
    'col_exp': "The preceding entity or the largest settlement of a polity.",
    'max_digits': 70,
    'null_meaning': "This polity did not have a preceding entity."
    },
    {'colname': "succeeding_entity",
    'dtype': MYTXT,
    'varname': 'politysucceedingentity', # key
    'col_exp': "The succeeding entity or the largest settlement of a polity.",
    'max_digits': 70,
    'null_meaning': "This polity did not have a succeeding entity."
    },
    {
    'colname': "supracultural_entity",
    'dtype': MYTXT,
    'varname': 'politysupraculturalentity', # key
    'col_exp': "The supracultural entity or the largest settlement of a polity.",
    'max_digits': 100,
    'null_meaning': "This polity did not have a supracultural entity."
    },
    {
    'colname': "scale_from",
    'dtype': MYINT,
    'varname': 'polityscaleofsupraculturalinteraction', # key
    'col_exp': "The lower scale of supra cultural interactionfor a polity.",
    'units': "km squared",
    'min': 0,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "scale_to",
    'dtype': MYINT,
    'varname': 'polityscaleofsupraculturalinteraction', # key
    'col_exp':  "The upper scale of supra cultural interactionfor a polity.",
    'units': "km squared",
    'min': 0,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "alternate_religion_genus",
    'dtype': MYTXT_CH,
    'varname': 'polityalternatereligiongenus', # key
    'col_exp': "The alternate religion genus of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_genus_CHOICES, "religion_genus")[0],
    'null_meaning': "This polity did not have a alternatereligion genus."
    },
    {
    'colname': "alternate_religion_family",
    'dtype': MYTXT_CH,
    'varname': 'polityalternatereligionfamily', # key
    'col_exp': "The alternate religion family of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_family_CHOICES, "religion_family")[0],
    'null_meaning': "This polity did not have a alternate religion family."
    },
    {
    'colname': "alternate_religion",
    'dtype': MYTXT_CH,
    'varname': 'polityalternatereligion', # key
    'col_exp': "The alternate religion of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_CHOICES, "religion")[0],
    'null_meaning': "This polity did not have a alternate religion ."
    },
    {
    'colname': "expert",
    'dtype': MYFOREIGN,
    'varname': 'polityexpert', # key
    'col_exp': "The expert of a polity.",
    'foreign_key': "SeshatExpert",
    'foreign_key_related_name': "seshat_expert",
    'null_meaning': "This polity did not have an expert."
    },
    {
    'colname': "editor",
    'dtype': MYFOREIGN,
    'varname': 'polityeditor', # key
    'col_exp': "The editor of a polity.",
    'foreign_key': "SeshatExpert",
    'foreign_key_related_name': "seshat_editor",
    'null_meaning': "This polity did not have an editor."
    },
    {
    'colname': "religious_tradition",
    'dtype': MYTXT,
    'varname': 'polityreligioustradition', # key
    'col_exp': "The details of religious traditions.",
    'max_digits': 100,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    }
]


In [ ]:
def var_col_dic_nmaker(ALL_GENERAL_VARS_LIST, ALL_GENERAL_COLS_LIST):
    mother_dic = {}
    for var_dic in ALL_GENERAL_VARS_LIST:
        new_entries = []
        for col_dic in ALL_GENERAL_COLS_LIST:
            if var_dic["varname"] == col_dic["varname"] and col_dic["colname"] not in new_entries:
                new_entries.append(col_dic["colname"])
        mother_dic[var_dic["varname"]] = new_entries
    return mother_dic

In [ ]:
my_dic = var_col_dic_nmaker(ALL_GENERAL_VARS_LIST, ALL_GENERAL_COLS_LIST)

In [ ]:
my_dic

In [ ]:
gen_var_mapper = {}
for var_old in ultimate_dic.keys():
    if var_old == 'peak_date':
        gen_var_mapper[var_old] = 'politypeakyears'
        continue
    elif var_old == 'ra':
        gen_var_mapper[var_old] = 'polityresearchassistant'
        continue
    elif var_old == 'alternative_names':
        gen_var_mapper[var_old] = 'polityalternativename'
        continue
    elif var_old == 'succeeding_(quasi)polity':
        gen_var_mapper[var_old] = 'politysucceedingentity'
        continue
    elif var_old == 'preceeding_(quasi)polity':
        gen_var_mapper[var_old] = 'politypreceedingentity'
        continue
    var_new = "polity" + var_old.replace("_", "").replace("(", "").replace(")", "").replace("-", "")
    gen_var_mapper[var_old] = var_new

In [ ]:
ultimate_dic.keys()

In [ ]:
def get_all_users_experts(pretty_ultimate_dic_clean):
    # CREATE the PeOLPE (USER and EXPERTS) tables:
    expert_mapper = {}
    for var_name, var_df in pretty_ultimate_dic_clean.items():
        if var_name in ["ra", "editor", "expert"]: 
            for index, item in var_df["values_df"].iterrows():
                if item[1] not in expert_mapper:
                    expert_mapper[item[2]] = index + 1
    return expert_mapper

a_mapping = get_all_users_experts(pretty_ultimate_dic)

In [ ]:
a_mapping

In [ ]:
a = "ali"

In [ ]:
b = a.split(" ")

In [ ]:

ALL_GENERAL_VARS_LIST =[
    {
    'varname': 'polityresearchassistant',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The RA(s) who worked on a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "Staff", # Hier 
    },
    {
    'varname': 'polityutmzone',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The UTM Zone of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityoriginalname',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The original name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternativename',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternative name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politypeakyears',
    'cols': 2,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternative name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityduration',
    'cols': 2,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternative name of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politydegreeofcentralization',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The degree of centralization of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politysuprapolityrelations',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The supra polity relations of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politycapital',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The capital or the largest settlement of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politylanguage',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The language or the largest settlement of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politylinguisticfamily',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The linguistic family of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politylanguagegenus',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The language genus of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligiongenus',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The religion genus of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligionfamily',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The religion family of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligion',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The religion of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityrelationshiptoprecedingentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The polity relationship to preceding (quasi)polity", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityprecedingentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The preceding entity of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politysucceedingentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The succeeding entity of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'politysupraculturalentity',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The supracultural entity of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityscaleofsupraculturalinteraction',
    'cols': 2,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The scale_of_supra_cultural_interaction of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternatereligiongenus',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternate religion genus of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternatereligionfamily',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternate religion family of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityalternatereligion',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The alternate religion  of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityexpert',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The expert of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityeditor',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The editor of a polity.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    },
    {
    'varname': 'polityreligioustradition',
    'cols': 1,
    'needsSeshatCommon': True,
    'db_name': "general", # Hier
    'main_desc': "The details of religious traditions.", # Hier
    'section': "General Variables", # Hier
    'subsection': "General", # Hier 
    }
]


ALL_GENERAL_COLS_LIST =[
{
    'colname': "polity_ra",
    'dtype': MYFOREIGN,
    'varname': 'polityresearchassistant', # key
    'col_exp': "The RA of a polity.",
    'foreign_key': "SeshatExpert",
    'foreign_key_related_name': "seshat_research_assistant",
    },
    {
    'colname': "utm_zone",
    'dtype': MYTXT,
    'varname': 'polityutmzone', # key
    'col_exp': "The details of UTM_ZONE.",
    'max_digits': 5,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "original_name",
    'dtype': MYTXT,
    'varname': 'polityoriginalname', # key
    'col_exp': "The details of original_name.",
    'max_digits': 100,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "alternative_name",
    'dtype': MYTXT,
    'varname': 'polityalternativename', # key
    'col_exp': "The details of alternative_name.",
    'max_digits': 100,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "peak_year_from",
    'dtype': MYINT,
    'varname': 'politypeakyears', # key
    'col_exp': "The beginning of the peak year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "peak_year_to",
    'dtype': MYINT,
    'varname': 'politypeakyears', # key
    'col_exp':  "The end of the peak year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "polity_year_from",
    'dtype': MYINT,
    'varname': 'polityduration', # key
    'col_exp': "The beginning year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "polity_year_to",
    'dtype': MYINT,
    'varname': 'polityduration', # key
    'col_exp':  "The end year for a polity.",
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "degree_of_centralization",
    'dtype': MYTXT_CH,
    'varname': 'politydegreeofcentralization', # key
    'col_exp': "The details of degree_of_centralization.",
    'max_digits': 50,
    'choices': tuple_choices_maker_2(degree_of_centralization_CHOICES, "degree_of_centralization")[0],
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "supra_polity_relations",
    'dtype': MYTXT_CH,
    'varname': 'politysuprapolityrelations', # key
    'col_exp': "The details of supra polity relations.",
    'max_digits': 50,
    'choices': tuple_choices_maker_2(supra_polity_relations_CHOICES, "supra_polity_relations")[0],
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "capital",
    'dtype': MYTXT,
    'varname': 'politycapital', # key
    'col_exp': "The capital or the largest settlement of a polity.",
    'max_digits': 70,
    'null_meaning': "This polity did not have a capital or the largest settlement."
    },
    {
    'colname': "language",
    'dtype': MYTXT_CH,
    'varname': 'politylanguage', # key
    'col_exp': "The language of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(language_CHOICES, "language")[0],
    'null_meaning': "This polity did not have a language."
    },
    {
    'colname': "linguistic_family",
    'dtype': MYTXT_CH,
    'varname': 'politylinguisticfamily', # key
    'col_exp': "The linguistic family of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(linguistic_family_CHOICES, "linguistic_family")[0],
    'null_meaning': "This polity did not have a linguistic family."
    },
    {
    'colname': "language_genus",
    'dtype': MYTXT_CH,
    'varname': 'politylanguagegenus', # key
    'col_exp': "The language genus of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(linguistic_family_CHOICES, "linguistic_family")[0],
    'null_meaning': "This polity did not have a language Genus."
    },
    {
    'colname': "religion_genus",
    'dtype': MYTXT_CH,
    'varname': 'polityreligiongenus', # key
    'col_exp': "The religion genus of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_genus_CHOICES, "religion_genus")[0],
    'null_meaning': "This polity did not have a religion genus."
    },
    {
    'colname': "religion_family",
    'dtype': MYTXT_CH,
    'varname': 'polityreligionfamily', # key
    'col_exp': "The religion family of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_family_CHOICES, "religion_family")[0],
    'null_meaning': "This polity did not have a religion family."
    },
    {
    'colname': "religion",
    'dtype': MYTXT_CH,
    'varname': 'polityreligion', # key
    'col_exp': "The religion of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_CHOICES, "religion")[0],
    'null_meaning': "This polity did not have a religion."
    },
    {
    'colname': "relationship_to_preceding_entity",
    'dtype': MYTXT_CH,
    'varname': 'polityrelationshiptoprecedingentity', # key
    'col_exp': "The polity relationship to preceding (quasi)polity",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(relationship_to_preceding_entity_CHOICES, "relationship_to_preceding_entity")[0],
    'null_meaning': "This polity did not have a relationship to preceding (quasi)polity"
    },
    {
    'colname': "preceding_entity",
    'dtype': MYTXT,
    'varname': 'polityprecedingentity', # key
    'col_exp': "The preceding entity or the largest settlement of a polity.",
    'max_digits': 70,
    'null_meaning': "This polity did not have a preceding entity."
    },
    {'colname': "succeeding_entity",
    'dtype': MYTXT,
    'varname': 'politysucceedingentity', # key
    'col_exp': "The succeeding entity or the largest settlement of a polity.",
    'max_digits': 70,
    'null_meaning': "This polity did not have a succeeding entity."
    },
    {
    'colname': "supracultural_entity",
    'dtype': MYTXT,
    'varname': 'politysupraculturalentity', # key
    'col_exp': "The supracultural entity or the largest settlement of a polity.",
    'max_digits': 100,
    'null_meaning': "This polity did not have a supracultural entity."
    },
    {
    'colname': "scale_from",
    'dtype': MYINT,
    'varname': 'polityscaleofsupraculturalinteraction', # key
    'col_exp': "The lower scale of supra cultural interactionfor a polity.",
    'units': "km squared",
    'min': 0,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "scale_to",
    'dtype': MYINT,
    'varname': 'polityscaleofsupraculturalinteraction', # key
    'col_exp':  "The upper scale of supra cultural interactionfor a polity.",
    'units': "km squared",
    'min': 0,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    },
    {
    'colname': "alternate_religion_genus",
    'dtype': MYTXT_CH,
    'varname': 'polityalternatereligiongenus', # key
    'col_exp': "The alternate religion genus of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_genus_CHOICES, "religion_genus")[0],
    'null_meaning': "This polity did not have a alternatereligion genus."
    },
    {
    'colname': "alternate_religion_family",
    'dtype': MYTXT_CH,
    'varname': 'polityalternatereligionfamily', # key
    'col_exp': "The alternate religion family of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_family_CHOICES, "religion_family")[0],
    'null_meaning': "This polity did not have a alternate religion family."
    },
    {
    'colname': "alternate_religion",
    'dtype': MYTXT_CH,
    'varname': 'polityalternatereligion', # key
    'col_exp': "The alternate religion of a polity.",
    'max_digits': 70,
    'choices': tuple_choices_maker_2(religion_CHOICES, "religion")[0],
    'null_meaning': "This polity did not have a alternate religion ."
    },
    {
    'colname': "expert",
    'dtype': MYFOREIGN,
    'varname': 'polityexpert', # key
    'col_exp': "The expert of a polity.",
    'foreign_key': "SeshatExpert",
    'foreign_key_related_name': "seshat_expert",
    'null_meaning': "This polity did not have an expert."
    },
    {
    'colname': "editor",
    'dtype': MYFOREIGN,
    'varname': 'polityeditor', # key
    'col_exp': "The editor of a polity.",
    'foreign_key': "SeshatExpert",
    'foreign_key_related_name': "seshat_editor",
    'null_meaning': "This polity did not have an editor."
    },
    {
    'colname': "religious_tradition",
    'dtype': MYTXT,
    'varname': 'polityreligioustradition', # key
    'col_exp': "The details of religious traditions.",
    'max_digits': 100,
    'null_meaning': "No_Value_Provided_in_Old_Wiki"
    }
]


In [ ]:
def convert_two_python_dics_to_python_vars_dic(list_of_dic_vars, list_of_dic_cols, my_db_name="general"):
    vars_dic_output = {}
    for v_name in list_of_dic_vars:
        # create a ,ain_ dic with all the avialable var items:
        v_top_dic = {}
        col_counter = 1
        for v_top_key, v_top_val in v_name.items():
            v_top_dic[v_top_key] = v_top_val
        for c_name in list_of_dic_cols:
            if v_name["varname"] == c_name["varname"]:
                good_key = "col" + str(col_counter)
                v_top_dic[good_key] = c_name
                col_counter = col_counter + 1
        vars_dic_output[v_name["varname"]] = v_top_dic

    return vars_dic_output

In [ ]:
aaaaaa = convert_two_python_dics_to_python_vars_dic(ALL_GENERAL_VARS_LIST, ALL_GENERAL_COLS_LIST)

In [ ]:
input_str = "5,677"
a_str = ":"
stripped_str = input_str.replace(",", "").strip()
my_catches = re.findall(fr'[\[]{{0,1}}[ ]{{0,1}}(\d+)[ ]{{0,1}}[-|_| ]{{0,1}}[ ]{{0,1}}([\d+]*)[ ]{{0,1}}[\]]{{0,1}}[ ]{{0,1}}[{a_str}]{{0,1}}(.*)', stripped_str)

----

In [ ]:
print(f"{'12345':,}")

In [ ]:
csv_df = pd.read_csv("referencess_AWS.csv", delimiter="|")
refs_aws = []
for index, row in enumerate(csv_df.iterrows()):
    refs_aws.append(row[1]['zotero_link'])
    
csv_df = pd.read_csv("referencess_local.csv", delimiter="|")
refs_local = []
for index, row in enumerate(csv_df.iterrows()):
    refs_local.append(row[1]['zotero_link'])

In [ ]:
refs_aws

In [ ]:
with open("Seshat_Databank_jan_23.json") as json_zotero:
    json_zotero_data_list = json.load(json_zotero)
for zot_data in json_zotero_data_list:
    if zot_data.get("id").split("/")[-1] not in refs_aws:
        print(zot_data.get("id").split("/")[-1], ": ", zot_data.get("title"))

In [ ]:
with open("Seshat_Databank_jan_23.json") as json_zotero:
    json_zotero_data_list = json.load(json_zotero)
for zot_data in json_zotero_data_list:
    if zot_data.get("id").split("/")[-1] not in refs_local:
        print(zot_data.get("id").split("/")[-1], ": ", zot_data.get("title"))

In [ ]:
hs_csv_df = pd.read_csv("human_sacrifices_download.csv")
hs_mapper_list = []
for index, row in enumerate(hs_csv_df.iterrows()):
    hs_mapper_list.append(row[1]['polity_old_ID'])

In [ ]:
if 0:
    print("Zero")
else:
    print("OOOps")

In [ ]:
polity_csv_df = pd.read_csv("./CSV_AWS/" +"my_politys.csv")
polity_mapper_dic = {}
for index, row in enumerate(polity_csv_df.iterrows()):
    polity_mapper_dic[row[1]['id']] = row[1]['name']

In [ ]:
polity_mapper_dic

In [ ]:
with open('hs_latest_insertion_AWS.sql') as f:
    lines = f.readlines()

In [ ]:
count = 1
pol_ids = []
pol_names = []
for line in lines:
    the_pol_id = line.split("(")[2].split(",")[0]
    pol_name = polity_mapper_dic[int(the_pol_id)]
    pol_ids.append(the_pol_id)
    pol_names.append(pol_name)
    count += 1

In [ ]:
for pol_name in pol_names:
    if pol_name not in hs_mapper_list:
        print(pol_name)

In [ ]:
def fix_hs(selected_lines):    
    with open(root_dir + "/hs_fixed_local.sql", "w") as gen_var_sql_file:
        all_rows_with_var_sql = []
        for index, line in enumerate(selected_lines):
            a_row = f"INSERT INTO crisisdb_human_sacrifice (id, first_name, last_name, username, email, password, is_superuser, is_staff, is_active, date_joined) VALUES ({user_id}, '{first_name}', '{last_name}', '{first_name}_{last_name}', '{first_name}@{last_name}.com', 'aliyaret_khoda_behamrahet', 'false', 'true', 'false', '2022-10-31 19:10:20.735+01');"
            all_rows_with_var_sql.append(a_row)

        gen_var_sql_file.write("\n".join(all_rows_with_var_sql))
        print(f"{len(all_rows_with_var_sql)} sql insertion rows added to: users_experts.sql")

# TODO:

- start from `ultimate_dic_without_duplicates_BACKUP_FRIDAY_JAN_6th` and make sure that all the zoteroID keys have a list of several items as the value.
- make sure that all the zoteros chosen are proper ones: for example check if the author is too short or if the author does not fit our chosen text
- then get all the refs that are still to get a proper zoter and see if you can improve the code to find more zoteros for the remianing ones.
- If the above is not fruitful, start making a final dictionary with the extra values out and the real meat (zotero codes + key, etc) in. 

In [ ]:
from main_scraper import *
from z_helpers import *
import time

In [ ]:
10+12

In [ ]:
#all_unique_refs_with_reps = ultimate_ref_dic_maker_duplicate_finder()

In [ ]:
start_time = time.time()

#my_ult_ref_dic, my_ult_meta_data = ultimate_ref_dic_maker("mother_dic_for_refs.json")
my_ult_ref_dic, my_ult_metadata = ultimate_ref_dic_maker_plus("mother_dic_with_duplicates.json")

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
my_ult_metadata

In [ ]:
my_ult_ref_dic['MAJIDBENAM_REF_10_AfDurrn']

In [ ]:
# this creates a new trimmed dic that needs to be merged with an ultimate dic:
go_find_zoteros("ultimate_dic_without_duplicates_TEST.json")

In [ ]:
x, meta_x = merge_trimmed_data_and_create_a_new_ultimate_dic("trimmed_dic_UPDATED_to_be_merged.json", "ultimate_dic_without_duplicates.json")

In [ ]:
meta_x

In [ ]:
a_dic = {
    "Id": [],
}
a_dic["Id"] = ["aliii",]

In [ ]:
a_dic

In [ ]:
# second round
go_find_zoteros_round_2("ultimate_dic_without_duplicates.json")

In [ ]:
x, meta_x = merge_trimmed_data_and_create_a_new_ultimate_dic("trimmed_dic_UPDATED_to_be_merged.json", "ultimate_dic_without_duplicates.json")

In [ ]:
meta_x

In [ ]:
go_find_zoteros_round_3("ultimate_dic_without_duplicates.json")

In [ ]:
x, meta_x = merge_trimmed_data_and_create_a_new_ultimate_dic("trimmed_dic_UPDATED_to_be_merged.json", "ultimate_dic_without_duplicates.json")

In [ ]:
meta_x

In [ ]:
x["MAJIDBENAM_REF_24_AfDurrn"]

In [ ]:
unique_trimmed_refs_needing_work = []
for kk, vv in x.items():
    if not vv["zoteroID"] and not vv["hasPersonalComment"] and vv["trimmedText"] and vv["trimmedText"] not in unique_trimmed_refs_needing_work:
        unique_trimmed_refs_needing_work.append(vv["trimmedText"])

In [ ]:
len(unique_trimmed_refs_needing_work)

In [ ]:
for i in unique_trimmed_refs_needing_work[:12]:
    print(i)
    print("______")


In [ ]:
"""
ROUND 1:
{'No_of_Unique_Refs': 28732,
 'No_of_Unique_trimmed_Refs': 17245,
 'No_of_Unique_trimmed_Refs_needing_work': 11170}
"""

In [ ]:
import pprint
bad_boys = []
for k, v in x.items():
    if not v.get("zoteroID"):
        bad_boys.append(v)
        if len(bad_boys) >= 10:
            break

In [ ]:
bad_boys[7]

In [ ]:
x['MAJIDBENAM_REF_1_AfDurrn']

In [ ]:
"ali yaret".split(";")

In [ ]:
my_ult_metadata

In [ ]:
with open("ultimate_dic_without_duplicates_BACKUP_FRIDAY_JAN_6th.json") as my_f:
    friday_data = json.load(my_f)


In [ ]:
all_semicolons = 0
for key, value in friday_data.items():
    raw_value_0 = value["trimmedText"]
    raw_value = raw_value_0.replace("&amp;", " ").replace("&nbsp;", " ")
    num_part_obj = re.search('(\d{4})', raw_value)
    if ";" in raw_value and (
        "an updated version is available online at" not in raw_value) and (
        "for a brief look at the decline" not in raw_value):
        raw_value_split = raw_value.split(";")
        pot_good = True
        for a_pot in raw_value_split:
            num_part_obj = re.search('(\d{4})', a_pot)
            if not num_part_obj:
                pot_good = False
                break
            elif int(num_part_obj.group(1)) >= 2023 or int(num_part_obj.group(1)) <= 1500:
                pot_good = False
                break
        if pot_good:
            if all_semicolons <= 40:
                print(key)
                for aa in raw_value_split:
                    print("* ", aa.strip())
                print("____")
            all_semicolons += 1

print(all_semicolons)

In [ ]:
all_variants = []
for key, value in friday_data.items():
    raw_value = value["zoteroID"]
    type_ID = type(raw_value)
    len_ID = len(raw_value)
    if len_ID > 0:
        inner_length = len(raw_value[0])
    else:
        inner_length = 0
    a_tuple = (len_ID, type_ID, inner_length)
    if inner_length >= 15:
        print(key, ":", value["trimmedText"],raw_value)
    if a_tuple not in all_variants:
        all_variants.append(a_tuple)

print(all_variants)

In [ ]:
len(my_ult_ref_dic)

In [ ]:
with open("Seshat_Databank_Dec_6.json") as json_zotero:
    json_zotero_data = json.load(json_zotero)

In [ ]:
json_zotero_data[77]

In [ ]:
12% 5

In [ ]:
my_ult_ref_dic['MAJIDBENAM_REF_142_AfGhurd']

In [ ]:
my_ult_ref_dic['MAJIDBENAM_REF_188_AfGhurd']

In [ ]:
import re

a_str = """
<sup id="cite_ref-1009" class="reference"><a href="#cite_note-1009">[1,009]</a></sup>
"""
#<sup id="cite_ref-1009" class="reference"><a href="#cite_note-1009">[1,009]</a></sup>


RefRegex = re.compile('<sup id="cite_ref-(\d{1,4})" class="reference"><a href="#cite_note-(\d{1,4})">\[(\d{1,4}|\d{1},\d{3})\]</a></sup>')
catches_all = RefRegex.finditer(a_str)

In [ ]:
catches_all

In [ ]:
for index, catches in enumerate(catches_all):
    updated_catch_3 = catches.group(3).replace(",", "")
    print(catches.group(1), catches.group(2), updated_catch_3, type(updated_catch_3))
    new_match_regex = f'<li id="cite_note-{catches.group(1)}"><span class="mw-cite-backlink"><a href="#cite_ref-{catches.group(1)}"><span class="cite-accessibility-label">Jump up </span>↑</a></span> <span class="reference-text">(.*)</span>'
    a_new_str = f"""hthtz
<li id="cite_note-1009"><span class="mw-cite-backlink"><a href="#cite_ref-1009"><span class="cite-accessibility-label">Jump up </span>↑</a></span> <span class="reference-text">(Higham 2014b, pp. 351-352)</span>
</li>"""
    RefRegex = re.compile(new_match_regex)
    catches_all_new = RefRegex.finditer(a_new_str)
    print(a_new_str)
    for c in catches_all_new:
        print(c.group(1))
    print(catches_all_new)

In [ ]:
a_new_str = """
<li id="cite_note-{catches.group(1)}"><span class="mw-cite-backlink"><a href="#cite_ref-{catches.group(1)}"><span class="cite-accessibility-label">Jump up </span>↑</a></span> <span class="reference-text">Ali yaret</span></li>
"""

In [ ]:
a_new_str = """
to the Heart of Islam pp.87-77"""
pp_finderRegex = re.compile(' pp.\s*(\d{2})-(\d{1,3})')
catches_all = list(pp_finderRegex.finditer(a_new_str))

In [ ]:
for pp_c in catches_all:
    to_be_removed = pp_c.group(1) + "-" + pp_c.group(2)
    print("JI:", to_be_removed)


In [ ]:
with open("all_english_words.txt", "r") as my_file:
    contents = my_file.readlines()

In [ ]:
for word in contents:
    word = word.lower()
    if len(word) == 6 and "a" not in word and "l" not in word and "e" not in word and "w" in word and word[0] != "w" and "h" in word and word[1]!= "h":
        print(word, end="")

In [ ]:
def children_finder(raw_value_0):
    """
    finds children ain asingle reference in a seshat wiki referece
    """
    raw_value = raw_value_0.replace("&amp;", " ").replace("&nbsp;", " ")
    num_part_obj = re.search('(\d{4})', raw_value)
    if ";" in raw_value and (
        "an updated version is available online at" not in raw_value) and (
        "for a brief look at the decline" not in raw_value):
        raw_value_split = raw_value.split(";")
        pot_children_goodness = []
        pot_good_children_list = []
        for a_pot_child in raw_value_split:
            num_part_obj = re.search('(\d{4})', a_pot_child)
            if not num_part_obj:
                pot_children_goodness.append(False)
            elif int(num_part_obj.group(1)) >= 2023 or int(num_part_obj.group(1)) <= 1500:
                pot_children_goodness.append(False)
            else:
                pot_children_goodness.append(True)
                pot_good_children_list.append(a_pot_child)
        # if the solit part sounds like a good boy:
        if len(pot_good_children_list) >= 2:
            return pot_good_children_list
        else:
            return [raw_value_0,]
    else:
        return [raw_value_0,]

In [ ]:
aaaaaa = children_finder("ali yarte 345")

In [ ]:
aaaaaa